## Scraping banten.antarnews.com

In [19]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
from urllib.request import Request, urlopen

In [20]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [21]:
def scrape_links(page_number):
    url = f"https://banten.antaranews.com/seputar-banten/{page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [22]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 9 links from page 1
Total Links: 9


In [23]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "tags-wrapper"})
            if author_elem:
                author_text = author_elem.find('span')
                author_text = author_text.text.split(':')[1]
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix"})
            if body_elem:
                 content_text = body_elem.text
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [24]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [25]:
df = pd.DataFrame(results)
print(len(results))
df.head(100)

9


,title,author,category,date,content,region,link
0,"Laris, sejumlah GOR di Tangerang tersewa hingg...",Achmad Irfan,Category not found,"Selasa, 26 September 2023 17:56 WIB",\n Tangerang (ANTARA) - Dinas Pem...,banten,https://banten.antaranews.com/berita/258573/la...
1,Polisi amankan tujuh orang pelaku perusakan Pa...,Azmi Syamsul Ma'arif,Category not found,"Selasa, 26 September 2023 16:08 WIB",\n Tangerang (ANTARA) - Kepolisia...,banten,https://banten.antaranews.com/berita/258549/po...
2,BPBD Lebak sudah distribusi air bersih 800.000...,Mansyur suryana,Category not found,"Selasa, 26 September 2023 20:21 WIB",\n Lebak (ANTARA) - Badan Penangg...,banten,https://banten.antaranews.com/berita/258600/bp...
3,Pemkot Tangerang sosialisasikan pendaftaran pe...,Achmad Irfan,Category not found,"Senin, 25 September 2023 22:41 WIB",\n Tangerang (ANTARA) - Pemerinta...,banten,https://banten.antaranews.com/berita/258417/pe...
4,Diskominfosatik Kabupaten Serang luncurkan apl...,Desi Purnama Sari,Category not found,"Selasa, 26 September 2023 21:07 WIB",\n Serang (ANTARA) - Dinas Komuni...,banten,https://banten.antaranews.com/berita/258609/di...
5,BPBD Tangerang temukan mayat diduga korban keb...,Azmi Syamsul Ma'arif,Category not found,"Senin, 25 September 2023 19:51 WIB",\n Tangerang (ANTARA) - Badan Pen...,banten,https://banten.antaranews.com/berita/258384/bp...
6,Kementerian Kesehatan optimalkan pelacakan kas...,Mansyur suryana,Category not found,"Selasa, 26 September 2023 17:02 WIB",\n Lebak (ANTARA) - Kementerian K...,banten,https://banten.antaranews.com/berita/258561/ke...
7,Selama Januari-September 2023 ada 99 kasus keb...,Mansyur suryana,Category not found,"Senin, 25 September 2023 20:00 WIB",\n Lebak (ANTARA) - Kebakaran di ...,banten,https://banten.antaranews.com/berita/258387/se...
8,Polisi Tangerang dalami soal surat permohonan ...,Azmi Syamsul Ma'arif,Category not found,"Selasa, 26 September 2023 20:18 WIB",\n Tangerang (ANTARA) - Kepolisia...,banten,https://banten.antaranews.com/berita/258618/po...


Tempat Simpan Save

In [26]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'./excel/banten_antranews_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')